In [7]:
import pandas as pd
import numpy as np

import datetime
import pickle

import openpyxl


In [8]:
shop_DE = pd.read_pickle("shop_de.pkl")
shop_DE.head(1)

,Monat,Produkt_A_DE,Produkt_B_DE,Produkt_C_DE,Gesamt_DE
0,January,15562.55,14774.46,148945.17,179282.18


In [9]:
shop_EU = pd.read_pickle("shop_eu.pkl")
shop_EU.head(1)

,Monat,Produkt_A_EU,Produkt_B_EU,Produkt_C_EU,Gesamt_EU
0,January,1514.89,15656.52,408530.28,425701.69


In [10]:
el_puente= pd.merge(shop_DE,shop_EU, on="Monat")
el_puente

,Monat,Produkt_A_DE,Produkt_B_DE,Produkt_C_DE,Gesamt_DE,Produkt_A_EU,Produkt_B_EU,Produkt_C_EU,Gesamt_EU
0,January,15562.55,14774.46,148945.17,179282.18,1514.89,15656.52,408530.28,425701.69
1,February,14055.57,15176.47,128859.95,158091.99,1495.04,35189.58,553895.17,590579.79
2,March,15633.91,18280.33,141602.71,175516.95,1788.99,60515.05,819422.67,881726.71
3,April,15792.85,19095.32,133780.49,168668.66,1392.18,79808.05,1015918.78,1097119.01
4,May,16947.49,20206.11,135993.78,173147.38,1719.59,104693.52,1257335.38,1363748.49
5,June,17082.75,20056.65,135206.77,172346.17,1588.10,122367.05,1425203.06,1549158.21
6,July,17965.03,22288.14,143531.78,183784.95,1813.78,151031.05,1650579.04,1803423.87
7,August,17720.17,21707.81,139714.63,179142.61,1587.99,177628.52,1874637.00,2053853.51
8,September,16870.28,21084.84,130178.03,168133.15,1495.50,198167.05,2000719.74,2200382.29
9,October,17579.24,21564.91,128966.72,168110.87,1836.12,227776.05,2296581.83,2526194.00


In [11]:
el_puente.to_excel("El_Puente.xlsx", engine='openpyxl', float_format='%.2f')

In [12]:
el_puente.to_pickle("total.pkl")


In [13]:
df = pd.read_pickle("total.pkl")

In [14]:
df =df.drop(["Gesamt_DE", "Gesamt_EU"], axis=1)

In [15]:
df["Monat"].replace({"January":"Jan", "February":"Feb", "March":"Mar", "April":"Apr",
                            "May":"Mai", "June":"Jun", "July":"Jul", "August":"Aug", "September":"Sep",
                            "October":"Okt", "November":"Nov", "December":"Dez"}, inplace=True)

In [16]:
shop_ohne_gesamt = pd.melt(df, id_vars='Monat', var_name='Produkt', value_name='Umsatz')
shop_ohne_gesamt

,Monat,Produkt,Umsatz
0,Jan,Produkt_A_DE,15562.55
1,Feb,Produkt_A_DE,14055.57
2,Mar,Produkt_A_DE,15633.91
3,Apr,Produkt_A_DE,15792.85
4,Mai,Produkt_A_DE,16947.49
...,...,...,...
67,Aug,Produkt_C_EU,1874637.00
68,Sep,Produkt_C_EU,2000719.74
69,Okt,Produkt_C_EU,2296581.83
70,Nov,Produkt_C_EU,2424958.15


In [17]:
shop_ohne_gesamt.to_pickle("total_melt.pkl")